In [ ]:
#!pip uninstall pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=4d7bae651f11fc0fc5a93fa36cd6f062183ef16ab25711ec3d0883844e1bead2
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark



**Alternating Least Squares (ALS)** es un algoritmo utilizado en sistemas de recomendación, especialmente para la factorización de matrices. Es una técnica que descompone una matriz grande y dispersa (como una matriz de usuario-producto en la que la mayoría de las entradas son desconocidas) en dos matrices más pequeñas y densas. Estas dos matrices representan las características latentes de los usuarios y los productos.



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from collections import Counter

# Crear una sesión de Spark
spark = SparkSession.builder.appName('MovieRecommender').getOrCreate()

# Cargar los datos
df_ratings = spark.read.csv("/content/rating.csv", header=True, inferSchema=True)
df_movies = spark.read.csv("/content/movie.csv", header=True, inferSchema=True)

# Unir los dataframes de ratings y movies
df_ratings = df_ratings.join(df_movies, on="movieId", how="inner")

# Seleccionar las columnas necesarias
df_ratings = df_ratings.select('userId', 'movieId', 'rating', 'title', 'genres')

# Dividir los datos en conjuntos de entrenamiento y prueba
train, test = df_ratings.randomSplit([0.8, 0.2], seed=42)

# Crear el modelo ALS
als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)

# Entrenar el modelo
model = als.fit(train)

# Evaluar el modelo usando RMSE
predictions = model.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")

# Evaluar el modelo usando MAE
evaluator_mae = RegressionEvaluator(
    metricName="mae",
    labelCol="rating",
    predictionCol="prediction"
)
mae = evaluator_mae.evaluate(predictions)
print(f"Mean Absolute Error = {mae}")

# Función para obtener las recomendaciones para un usuario específico
def get_recommendations(user_id, model, df_movies, num_recommendations=10):
    # Obtener las recomendaciones para el usuario especificado
    recs = model.recommendForUserSubset(spark.createDataFrame([(user_id,)], ["userId"]), num_recommendations)
    recs = recs.collect()[0]['recommendations']
    movie_ids = [rec.movieId for rec in recs]

    # Obtener los detalles de las películas recomendadas
    recommended_movies = df_movies.filter(df_movies.movieId.isin(movie_ids)).select('title', 'genres').collect()

    recommendations = []
    for movie in recommended_movies:
        recommendations.append((movie['title'], movie['genres']))

    return recommendations

# Función para resumir los géneros de las recomendaciones
def summarize_genres(recommendations):
    genre_counter = Counter()
    for title, genres in recommendations:
        genre_list = genres.split('|')
        genre_counter.update(genre_list)

    return genre_counter

# Función para mostrar las recomendaciones de un usuario específico y resumir sus géneros preferidos
def show_user_recommendations_and_genre_summary(user_id):
    recommendations = get_recommendations(user_id, model, df_movies)
    print(f"Recomendaciones para el usuario {user_id}:")
    for title, genres in recommendations:
        print(f"Pelicula: {title}, Géneros: {genres}")

    genre_summary = summarize_genres(recommendations)
    print("\nResumen de géneros favoritos del usuario:")
    for genre, count in genre_summary.items():
        print(f"Género: {genre}, Conteo: {count}")

# Ejemplo de uso
user_id = 148  # Cambia esto por el ID del usuario que deseas recomendar
show_user_recommendations_and_genre_summary(user_id)


Root-mean-square error = 0.8122131260384973
Mean Absolute Error = 0.6335012833014136
Recomendaciones para el usuario 148:
Pelicula: Letter From Death Row, A (1998), Géneros: Crime|Drama
Pelicula: Twice Upon a Time (1983), Géneros: Animation|Fantasy
Pelicula: Hotel Terminus: The Life and Times of Klaus Barbie (Hôtel Terminus) (1988), Géneros: Documentary
Pelicula: Return to the 36th Chamber (Shao Lin da peng da shi) (1980), Géneros: Action|Comedy
Pelicula: Punk's Not Dead (2007), Géneros: Documentary
Pelicula: Johnny Cash at Folsom Prison (2008), Géneros: Documentary
Pelicula: Child's Christmas in Wales, A (1987), Géneros: Children|Drama
Pelicula: Mimino (1977), Géneros: Comedy
Pelicula: London - The Modern Babylon (2012), Géneros: Documentary
Pelicula: Magic of Ordinary Days, The (2005), Géneros: Drama

Resumen de géneros favoritos del usuario:
Género: Crime, Conteo: 1
Género: Drama, Conteo: 3
Género: Animation, Conteo: 1
Género: Fantasy, Conteo: 1
Género: Documentary, Conteo: 4
Género

Identifica y justifica las métricas de evaluación utilizadas para evaluar el desempeño de los sistemas de recomendación, con el proyecto elegido por equipo.

Root-mean-square error = 0.8122131260384973
Mean Absolute Error = 0.6335012833014136

Error Cuadrático Medio (RMSE)

El RMSE mide la magnitud promedio de los errores de predicción del modelo, tomando la raíz cuadrada de la media de los errores cuadráticos.

Interpretación:

Bajo RMSE: Indica que las predicciones del modelo están cerca de los valores reales, lo que significa un mejor rendimiento.
Alto RMSE: Indica que las predicciones del modelo están lejos de los valores reales, lo que significa un peor rendimiento.

Error Absoluto Medio (MAE)

El MAE mide la magnitud promedio de los errores de predicción del modelo, tomando el promedio de las diferencias absolutas entre los valores reales y los valores predichos.

Interpretación:

Bajo MAE: Indica que, en promedio, las predicciones del modelo están cerca de los valores reales.

Alto MAE: Indica que, en promedio, las predicciones del modelo están lejos de los valores reales.

Interpretación de los resultados:

RMSE = 0.86: Esto significa que la desviación estándar de los errores de predicción es 0.86. Es decir, en promedio, las predicciones del modelo están a 0.86 unidades de calificación de los valores reales.
MAE = 0.68: Esto significa que, en promedio, las predicciones del modelo están a 0.68 unidades de calificación de los valores reales.
Estos resultados indican que el modelo tiene una precisión moderada en sus predicciones. Dependiendo del contexto y los requisitos del sistema de recomendación, estos errores pueden ser aceptables o pueden indicar que se necesita mejorar el modelo.





Resultados y análisis. Enlista la menos 3 recomendaciones donde se muestren los resultados obtenidos del punto 1 (implementación del algoritmo). La evidencia se debe pone en el repositorio GitHub del equipo.

In [ ]:
user_id = 9032  # Cambia esto por el ID del usuario que deseas recomendar
show_user_recommendations_and_genre_summary(user_id)

Recomendaciones para el usuario 9032:
Pelicula: Class Trip, The (La classe de neige) (1998), Géneros: Drama|Mystery
Pelicula: Myra Breckinridge (1970), Géneros: Comedy
Pelicula: Black Girl (La noire de...) (1966), Géneros: Drama
Pelicula: Bob Funk (2009), Géneros: Comedy|Romance
Pelicula: Hamoun (1990), Géneros: Drama
Pelicula: Christine Jorgensen Story, The (1970), Géneros: Drama
Pelicula: Taming the Fire (Ukroshcheniye ognya) (1972), Géneros: Documentary|Drama|Sci-Fi
Pelicula: Whoopi Goldberg Presents Moms Mabley (2013), Géneros: Documentary
Pelicula: 2013 Rock and Roll Hall of Fame Induction Ceremony, The (2013), Géneros: Documentary|Musical
Pelicula: Bill Maher... But I'm Not Wrong (2010), Géneros: Comedy

Resumen de géneros favoritos del usuario:
Género: Drama, Conteo: 5
Género: Mystery, Conteo: 1
Género: Comedy, Conteo: 3
Género: Romance, Conteo: 1
Género: Documentary, Conteo: 3
Género: Sci-Fi, Conteo: 1
Género: Musical, Conteo: 1


In [ ]:
user_id = 5  # Cambia esto por el ID del usuario que deseas recomendar
show_user_recommendations_and_genre_summary(user_id)

Recomendaciones para el usuario 5:
Pelicula: Letter From Death Row, A (1998), Géneros: Crime|Drama
Pelicula: Smashing Pumpkins: Vieuphoria (1994), Géneros: Documentary|Musical
Pelicula: Hotel Terminus: The Life and Times of Klaus Barbie (Hôtel Terminus) (1988), Géneros: Documentary
Pelicula: Punk's Not Dead (2007), Géneros: Documentary
Pelicula: Pearl Jam: Immagine in Cornice - Live in Italy 2006 (2007), Géneros: Documentary|Musical
Pelicula: Hatful of Rain, A (1957), Géneros: Drama
Pelicula: Toys in the Attic (1963), Géneros: Drama
Pelicula: Johnny Cash at Folsom Prison (2008), Géneros: Documentary
Pelicula: Tis kakomoiras (1963), Géneros: Comedy
Pelicula: Afstiros katallilo (2008), Géneros: Comedy

Resumen de géneros favoritos del usuario:
Género: Crime, Conteo: 1
Género: Drama, Conteo: 3
Género: Documentary, Conteo: 5
Género: Musical, Conteo: 2
Género: Comedy, Conteo: 2


In [ ]:
user_id = 105  # Cambia esto por el ID del usuario que deseas recomendar
show_user_recommendations_and_genre_summary(user_id)

Recomendaciones para el usuario 105:
Pelicula: Cabeza de Vaca (1991), Géneros: Action|Adventure
Pelicula: Smashing Pumpkins: Vieuphoria (1994), Géneros: Documentary|Musical
Pelicula: 7 Men from Now (Seven Men from Now) (1956), Géneros: Western
Pelicula: Motel, The (2005), Géneros: Comedy|Drama
Pelicula: Filth and Wisdom (2008), Géneros: Comedy|Drama|Musical|Romance
Pelicula: Hatful of Rain, A (1957), Géneros: Drama
Pelicula: Bob Funk (2009), Géneros: Comedy|Romance
Pelicula: Poolboy: Drowning Out the Fury (2011), Géneros: Comedy
Pelicula: Barfi! (2012), Géneros: Comedy|Drama|Romance
Pelicula: Bambi Meets Godzilla (1969), Géneros: Animation|Comedy

Resumen de géneros favoritos del usuario:
Género: Action, Conteo: 1
Género: Adventure, Conteo: 1
Género: Documentary, Conteo: 1
Género: Musical, Conteo: 2
Género: Western, Conteo: 1
Género: Comedy, Conteo: 6
Género: Drama, Conteo: 4
Género: Romance, Conteo: 3
Género: Animation, Conteo: 1
